In [127]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math as mt
import os
import pickle

In [128]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [129]:
df=pd.read_csv("data/labl.csv")
df.head()


,image,class,is_valid
0,ISIC_0024310_0,0,False
1,ISIC_0024310_1,0,False
2,ISIC_0024310_2,0,False
3,ISIC_0024310_3,0,False
4,ISIC_0024310_4,0,False


In [130]:
batch_size=10

In [131]:
def train_valid_test(pathcsv,trainprop,validprop,testprop,bs=batch_size):
    df=pd.read_csv(pathcsv)
    df=df.sample(frac=1).reset_index()
    x=df["is_valid"].value_counts()
    tot={i:[] for i in range(7)}
    x_train=[]
    x_test=[]
    x_valid=[]
    y_train=[]
    y_valid=[]
    y_test=[]
    print((x[0]//7))
    train_amnt=((int(trainprop*(x[0]//7))//bs)*bs)
    print(train_amnt)
    valid_amnt=int((int(((x[0]//7)-train_amnt)*validprop)//bs)*bs)
    test_amnt=int((int(x[1]*testprop)//bs)*bs)
    print(test_amnt,int(x[1]*testprop))
    for i,row in df.iterrows():
        if row["is_valid"]==False:
            tot[int(row["class"])].append(row["image"]+".jpg")
        elif test_amnt>0:
            x_test.append(row["image"]+".jpg")
            y_test.append(int(row["class"]))
            test_amnt-=1
        
    
    for v in tot:
        x_train+=(tot[v][:train_amnt])
        y_train+=[v]*train_amnt
        x_valid+=(tot[v][train_amnt:train_amnt+valid_amnt])
        y_valid+=[v]*valid_amnt
    return x_train,y_train,x_valid,y_valid,x_test,y_test

In [133]:
pathcsv = "data/labl.csv"
x_train,y_train,x_valid,y_valid,x_test,y_test=train_valid_test(pathcsv,0.75,0.75,0.76)

6035
4520
750 758


In [134]:
print(len(x_train),len(y_train),len(x_valid),len(y_valid),len(x_test),len(y_test))

31640 31640 7910 7910 750 750


In [135]:
print(x_test)

['ISIC_0026230.jpg', 'ISIC_0027514.jpg', 'ISIC_0025842.jpg', 'ISIC_0025965.jpg', 'ISIC_0029109.jpg', 'ISIC_0027687.jpg', 'ISIC_0025465.jpg', 'ISIC_0032533.jpg', 'ISIC_0024940.jpg', 'ISIC_0031389.jpg', 'ISIC_0033409.jpg', 'ISIC_0033843.jpg', 'ISIC_0032520.jpg', 'ISIC_0025921.jpg', 'ISIC_0027086.jpg', 'ISIC_0028011.jpg', 'ISIC_0025610.jpg', 'ISIC_0025504.jpg', 'ISIC_0028392.jpg', 'ISIC_0033238.jpg', 'ISIC_0033537.jpg', 'ISIC_0025972.jpg', 'ISIC_0032642.jpg', 'ISIC_0032062.jpg', 'ISIC_0026420.jpg', 'ISIC_0027772.jpg', 'ISIC_0033544.jpg', 'ISIC_0031464.jpg', 'ISIC_0029978.jpg', 'ISIC_0030474.jpg', 'ISIC_0030197.jpg', 'ISIC_0033950.jpg', 'ISIC_0030780.jpg', 'ISIC_0031933.jpg', 'ISIC_0029034.jpg', 'ISIC_0028003.jpg', 'ISIC_0027551.jpg', 'ISIC_0026291.jpg', 'ISIC_0024920.jpg', 'ISIC_0029025.jpg', 'ISIC_0032931.jpg', 'ISIC_0027250.jpg', 'ISIC_0029981.jpg', 'ISIC_0025276.jpg', 'ISIC_0032448.jpg', 'ISIC_0026412.jpg', 'ISIC_0029865.jpg', 'ISIC_0031450.jpg', 'ISIC_0031046.jpg', 'ISIC_0028131.jpg',

In [136]:
def find_suitable(h,w,mxval):
    gcd=mt.gcd(h,w)
    val=1
    while(val<=gcd):
        if h%val==0 and w%val==0 and (h//val)*(w/val)<=mxval:
            # print(val)
            return h//val,w//val
        val+=1

In [137]:
rh=None
rw=None

In [138]:
def preprocess_image(path,mxval):
  global rh
  global rw
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=1)
  sh=image.shape
  if rh==None and rw==None:
    rh,rw=find_suitable(sh[0],sh[1],mxval)
  image = tf.image.resize(image, [rh,rw])
  image=tf.squeeze(image)
  sh=image.shape
  # print(sh)
  image=image.numpy()
  numdig=len(str(sh[0]*sh[1]))
  mv=255.0*((10)**numdig)+sh[0]*sh[1]
  cnt=1
  for i, v in enumerate(image):
    for j,val in enumerate(v):
      image[i][j]=image[i][j]*((10)**numdig)+cnt
      cnt+=1
  
  image/=mv
  image=np.float32(image)
  # print(image.shape)
  # image=image.reshape(rw*rh,)
  # print(image.shape)
  return image

In [139]:
print(x_test[0])

ISIC_0026230.jpg


In [ ]:
# for i in range(5):
#     image=preprocess_image("data/balanced_image/"+'ISIC_0033544.jpg',200)

(9, 12)
(108,)


In [140]:
# image=preprocess_image("data/balanced_image/ISIC_0033544.jpg",200)
x1=[]
x2=[]
x3=[]
y1=[]
y2=[]
y3=[]

for i,v in enumerate(x_train):
    try:
        # print("data/balanced_image/"+x_train[i])
        x1.append(preprocess_image("data/balanced_image/"+x_train[i],200))
        y1.append(y_train[i])
    except Exception as e:
        print(x_train[i],y_train[i])
        
for i,v in enumerate(x_valid):
    try:
        x2.append(preprocess_image("data/balanced_image/"+x_valid[i],200))
        y2.append(y_valid[i])
    except Exception as e:
        print(x_valid[i],y_valid[i])
for i,v in enumerate(x_test):
    try:
        x3.append(preprocess_image("data/balanced_image/"+x_test[i],200))
        y3.append(y_test[i])
    except Exception as e:
        print(x_test[i],y_test[i])
    

In [ ]:
def oh(arr,depth=7):
     return tf.one_hot(arr, depth)
     

In [ ]:
# x1=np.array(x1)
# y1=np.float32(np.array(oh(y1)))
# x2=np.array(x2)
# y2=np.float32(np.array(oh(y2)))
# x3=np.array(x3)
# y3=np.float32(np.array(oh(y3)))

In [ ]:
x1=np.array(x1)
y1=np.float32(np.array(y1))
x2=np.array(x2)
y2=np.float32(np.array(y2))
x3=np.array(x3)
y3=np.float32(np.array(y3))

In [ ]:
y1.shape

(420,)

In [ ]:
# for i in range(10):
#     x1.append(preprocess_image("data/balanced_image/"+'ISIC_0033544.jpg',200))
#     y1.append(1)
#     x2.append(preprocess_image("data/balanced_image/"+'ISIC_0033544.jpg',200))
#     y2.append(1)

In [ ]:
x1.shape

(420, 9, 12)

In [ ]:
with open('data/train/x_train.pkl','wb') as f:
        pickle.dump(x1, f)
with open('data/train/y_train.pkl','wb') as f:
        pickle.dump(y1, f)
with open('data/valid/x_valid.pkl','wb') as f:
        pickle.dump(x2, f)
with open('data/valid/y_valid.pkl','wb') as f:
        pickle.dump(y2, f)
with open('data/test/x_test.pkl','wb') as f:
        pickle.dump(x3, f)
with open('data/test/y_test.pkl','wb') as f:
        pickle.dump(y3, f)